# MACHINE LEARNING
## AUTHOR: ANTE DUJIC
<hr style="border:2px solid gray"> </hr>

<center><img src="https://learn.microsoft.com/en-us/windows/ai/images/winml-graphic.png" width="800"/></center>

Machine learning is programming computers to optimize a performance criterion using example data or past experience . We have a model defined up to some parameters, and learning is the execution of a computer program to optimize the parameters of the model using the training data or past experience. The model may be predictive to make predictions in the future, or descriptive to gain knowledge from data.[1] Some implementations of machine learning use data and neural networks in a way that mimics the working of a biological brain. [2] Human brain cells, called neurons, form a complex, highly interconnected network and send electrical signals to each other to help humans process information. Similarly, an artificial neural network is made of artificial neurons that work together to solve a problem. Artificial neurons are software modules, called nodes, and artificial neural networks are software programs or algorithms that, at their core, use computing systems to solve mathematical calculations.[3]

### SINGLE NEURON
***

An artificial neuron (also referred to as a perceptron) is a mathematical function. It takes one or more inputs that are multiplied by values called “weights” and added together. This value is then passed to a non-linear function, known as an activation function, to become the neuron’s output. [4]

![](img/single_neuron.drawio.svg)

#### SINGLE INPUT
***

![](img/single_input.drawio.svg)

- INPUT FUNCTIONS

#### GENERATE DATA

#### TRAINING BY HAND

#### KERAS

### REFERENCES

1. https://www.geeksforgeeks.org/introduction-machine-learning/
2. https://en.wikipedia.org/wiki/Machine_learning
3. https://aws.amazon.com/what-is/neural-network/
4. https://srnghn.medium.com/deep-learning-overview-of-neurons-and-activation-functions-1d98286cf1e4